In [ ]:
import sqlite3
import numpy as np
import pandas as pd
import missingno as msno
from copy import deepcopy
from tqdm.auto import tqdm

import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster

import plotly.express as px
from plotly.offline import *
init_notebook_mode(connected = True)

In [ ]:
# Connect to the database 
conn = sqlite3.connect("kijiji_real_estate_gta.db", isolation_level=None,
                       detect_types=sqlite3.PARSE_COLNAMES)

# convert the database into a pandas data frame.
df = pd.read_sql_query("SELECT * FROM properties", conn)


In [ ]:
# As database is storing null value as N/A converting N/A to null value, replacing 'N/A' with np.nan
df.replace('N/A', np.nan, inplace=True)

#### Data Scrubbing

Missingness Analysis: 

In [ ]:
# get all columns with missing values.

col_mis_val = df.columns[df.isnull().sum() > 0]
col_mis_val = col_mis_val.to_list()

# visualize missing values with missingno.

msno.matrix(df[col_mis_val])

Data Completeness per column

In [ ]:
msno.bar(df[col_mis_val])

In [ ]:
# Get all columns whose missing values are greater than 50%
to_be_dropeed = df[col_mis_val].columns[df[col_mis_val].isna().mean() * 100 > 50]
print(f"More than 50% missing values:\n{list(to_be_dropeed)}")

In [ ]:
# drop all rows with missing address 
df.dropna(subset=['address'], inplace=True)

print(f"Missing Values in address : {df.address.isnull().sum()}")

Missing at Random Check:

In [ ]:
# lets check of pets_friendly 
df["pets_friendly"].value_counts()

Here we can see that data consists of only `Yes` records, so `No` record is intentionally left blank. We can add `No` to all remaining null rows. 

In [ ]:
df["pets_friendly"] = df["pets_friendly"].fillna("No")
df["pets_friendly"].value_counts()

#### Feature Engineering

Converting price to proper format.

In [ ]:
# Replace 'Please Contact' with np.nan in the price column as some listing contains please contact info. 
df['price'].replace('Please Contact', np.nan, inplace=True)

# Drop rows with NaN in the price column
df.dropna(subset=['price'], inplace=True)

# Remove non-numeric characters from the price and convert to float, trying to remove $
df['price'] = df['price'].str.replace(r'[^0-9.]', '', regex=True).astype(float)

In [ ]:
# fe_df = deepcopy(df)
# fe_df.columns

# fe_df = fe_df[["price"]]


# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer
# from sklearn.preprocessing import LabelEncoder
# from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, mean_absolute_error
# import warnings 
# warnings.filterwarnings("ignore")



# def impute_categorical_missing_data(passed_col):
    
#     df_null = df[df[passed_col].isnull()]
#     df_not_null = df[df[passed_col].notnull()]

#     X = df_not_null.drop(passed_col, axis=1)
#     y = df_not_null[passed_col]
    
#     other_missing_cols = [col for col in missing_data_cols if col != passed_col]
    
#     label_encoder = LabelEncoder()

#     for col in X.columns:
#         if X[col].dtype == 'object' or X[col].dtype == 'category':
#             X[col] = label_encoder.fit_transform(X[col])

#     if passed_col in bool_cols:
#         y = label_encoder.fit_transform(y)
        
#     iterative_imputer = IterativeImputer(estimator=RandomForestRegressor(random_state=42), add_indicator=True)

#     for col in other_missing_cols:
#         if X[col].isnull().sum() > 0:
#             col_with_missing_values = X[col].values.reshape(-1, 1)
#             imputed_values = iterative_imputer.fit_transform(col_with_missing_values)
#             X[col] = imputed_values[:, 0]
#         else:
#             pass
    
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     rf_classifier = RandomForestClassifier()

#     rf_classifier.fit(X_train, y_train)

#     y_pred = rf_classifier.predict(X_test)

#     acc_score = accuracy_score(y_test, y_pred)

#     print("The feature '"+ passed_col+ "' has been imputed with", round((acc_score * 100), 2), "accuracy\n")

#     X = df_null.drop(passed_col, axis=1)

#     for col in X.columns:
#         if X[col].dtype == 'object' or X[col].dtype == 'category':
#             X[col] = label_encoder.fit_transform(X[col])

#     for col in other_missing_cols:
#         if X[col].isnull().sum() > 0:
#             col_with_missing_values = X[col].values.reshape(-1, 1)
#             imputed_values = iterative_imputer.fit_transform(col_with_missing_values)
#             X[col] = imputed_values[:, 0]
#         else:
#             pass
                
#     if len(df_null) > 0: 
#         df_null[passed_col] = rf_classifier.predict(X)
#         if passed_col in bool_cols:
#             df_null[passed_col] = df_null[passed_col].map({0: False, 1: True})
#         else:
#             pass
#     else:
#         pass

#     df_combined = pd.concat([df_not_null, df_null])
    
#     return df_combined[passed_col]

# def impute_continuous_missing_data(passed_col):
    
#     df_null = df[df[passed_col].isnull()]
#     df_not_null = df[df[passed_col].notnull()]

#     X = df_not_null.drop(passed_col, axis=1)
#     y = df_not_null[passed_col]
    
#     other_missing_cols = [col for col in missing_data_cols if col != passed_col]
    
#     label_encoder = LabelEncoder()

#     for col in X.columns:
#         if X[col].dtype == 'object' or X[col].dtype == 'category':
#             X[col] = label_encoder.fit_transform(X[col])
    
#     iterative_imputer = IterativeImputer(estimator=RandomForestRegressor(random_state=42), add_indicator=True)

#     for col in other_missing_cols:
#         if X[col].isnull().sum() > 0:
#             col_with_missing_values = X[col].values.reshape(-1, 1)
#             imputed_values = iterative_imputer.fit_transform(col_with_missing_values)
#             X[col] = imputed_values[:, 0]
#         else:
#             pass
    
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     rf_regressor = RandomForestRegressor()

#     rf_regressor.fit(X_train, y_train)

#     y_pred = rf_regressor.predict(X_test)

#     print("MAE =", mean_absolute_error(y_test, y_pred), "\n")

#     X = df_null.drop(passed_col, axis=1)

#     for col in X.columns:
#         if X[col].dtype == 'object' or X[col].dtype == 'category':
#             X[col] = label_encoder.fit_transform(X[col])

#     for col in other_missing_cols:
#         if X[col].isnull().sum() > 0:
#             col_with_missing_values = X[col].values.reshape(-1, 1)
#             imputed_values = iterative_imputer.fit_transform(col_with_missing_values)
#             X[col] = imputed_values[:, 0]
#         else:
#             pass
                
#     if len(df_null) > 0: 
#         df_null[passed_col] = rf_regressor.predict(X)
#     else:
#         pass

#     df_combined = pd.concat([df_not_null, df_null])
    
#     return df_combined[passed_col]



Converting address to longitude and latitude. 

In [ ]:
import re
import pgeocode
import pandas as pd
from postalcodes_ca import fsa_codes

class PostalCodeProcessor:
    def __init__(self, df, address_column):
        self.df = df
        self.address_column = address_column
        self.nomi = pgeocode.Nominatim('CA')
        self.fsa_codes = fsa_codes

    def extract_postal_codes(self, address):
        pattern = r'\b[A-Z]\d[A-Z][ -]?\d[A-Z]\d\b'
        match = re.search(pattern, address)
        return match.group() if match else None

    def format_canadian_postal_code(self, postal_code):
        pattern = r'^([A-Z]\d[A-Z])?(\s)?(\d[A-Z]\d)$'
        match = re.search(pattern, postal_code.upper())
        if match:
            return f"{match.group(1)} {match.group(3)}" if match.group(2) != '\s' else postal_code.upper()
        return None

    def process_dataframe(self):
        result = []
        for address in self.df[self.address_column]:
            postal_code = self.extract_postal_codes(address)
            if postal_code:
                formatted_postal_code = self.format_canadian_postal_code(postal_code)
                fsa = formatted_postal_code.split()[0]

                try:

                    fsa_result = self.fsa_codes[fsa]
                    city = fsa_result.name.split("(")[0].strip()
                    query = self.nomi.query_postal_code(formatted_postal_code)
                    lat, lon = query["latitude"], query["longitude"]
                    result.append([lat, lon, city])

                except Exception as e:
                    result.append([None, None, None])

            else:
                result.append([None, None, None])
        
        # Add new columns to the dataframe
        self.df['latitude'] = [r[0] for r in result]
        self.df['longitude'] = [r[1] for r in result]
        self.df['city'] = [r[2] for r in result]
        
        return self.df

processor = PostalCodeProcessor(df, "address")

# add longitude, latitude and city in dataframe
df = processor.process_dataframe()

In [ ]:
# drop all the rows with either of the missing values.
df.dropna(subset=['latitude', 'longitude', 'city'], inplace=True)


#### EDA

Analyzing the price columns. 

In [ ]:
# Create a box plot for the price column
fig = px.box(df, y='price', title='Box Plot of Listing Prices', labels={'price': 'Price'}, template='plotly_dark')

fig.update_layout(
    height=800
)

fig.show()

In [ ]:
# Calculate IQR
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.90)
IQR = Q3 - Q1

# Filter out outliers
df = df[(df['price'] >= (Q1 - 1.5 * IQR)) & (df['price'] <= (Q3 + 1.5 * IQR))]


fig = px.box(df, y='price', title='Box Plot of Listing Prices', labels={'price': 'Price'}, template='plotly_dark')

fig.update_layout(
    height=800
)

fig.show()

Visualization of the listing distribution. 

In [ ]:
latitude = df["latitude"].mean()
longitude = df["longitude"].mean()
map = folium.Map(location=[latitude, longitude], zoom_start=11)

marker_cluster = MarkerCluster().add_to(map)

for _, row in df.iterrows():
    folium.Marker(location=[row["latitude"], row["longitude"]], 
                  icon=folium.Icon(color='black',icon_color='#FF0000')).add_to(marker_cluster)

map

**Top 10 city with highest listings.**

In [ ]:
# get listings per city
listing_per_city = df.city.value_counts().to_dict()

# convert dict to list
city = list(listing_per_city.keys())[:10]
count = list(listing_per_city.values())[:10]


fig = px.bar(x=city, y=count, text=count, template='plotly_dark')
# fig.update_traces(texttemplate="%{y}", textposition="auto")

fig.update_layout(
    title='Top 10 cities with property listing.',
    xaxis_title='city',
    yaxis_title='Number of listings.',
    xaxis={'categoryorder':'total descending'},
    height=800
)

fig.show()

**Top 10 Average price per city**

In [ ]:
avg_price_per_city = df.groupby('city')['price'].mean().reset_index()

avg_price_per_city = avg_price_per_city.sort_values(by='price', ascending=False)[:10]

fig = px.bar(avg_price_per_city, x='city', y='price', title='Average Listing Price per City', labels={'price': 'Average Price', 'city': 'City'},text=avg_price_per_city["price"], template='plotly_dark')

fig.update_layout(
    title='Average price in top 10 cities.',
    xaxis_title='City',
    yaxis_title='Average Listing price.',
    xaxis={'categoryorder':'total descending'},
    height=800
)

fig.show()


**Heatmap of price distribution:**

In [ ]:
lats_longs_weight = [[x[-1]["latitude"],x[-1]["longitude"],x[-1]["price"]] for x in df.iterrows()]

In [ ]:
map_obj = folium.Map(location=[latitude, longitude] , zoom_start = 9)

HeatMap(lats_longs_weight).add_to(map_obj)

map_obj